### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2023 ###

### Final Course Project: Due 5/5/23

# Project title goes here #

#### Lauren Ciha lciha@wisc.edu

*Disclaimer: I was not paid by Todoist to do my project on their app or to market their app in any way. I chose to do my project on the Todoist app and task dependency because I wanted to hack my to-do list, which is currently in Todoist. Many of the conclusions here transfer to other productivity apps as well.*

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

To-do lists are great for keeping track of what to do. But they rarely give you a place to start.

I use the Todoist app (put a link in here) to keep track of what I need to do. The app allows you to sort tasks into projects and further into sections. From there, you can set deadlines, add tags to sort tasks, and rate their priority.

Here's an example of some of my weekly tasks this semester:

![todolist.png](todolist.png)

You can see that I have laid out my tasks, put them in a section, and added tags estimating the amount of time it will take me to complete each task. One of these tasks has a subtask.

![subtask.png](subtask.png)

For this assignment, I prefer to print out the worksheet, fill it out, and then upload it as a pdf. In order to do my homework assignment, then, I need to *first* print out the worksheet. Normally, a subtask indicates part of a task that is included in the main task, but in this case, I'm using a subtask to express that a task depends on another, in other words, that I need to finish the subtask before marking the main task as complete.

Currently, Todoist doesn't have a way to label a task as dependent on another. In addition to subtasks, I sometimes express dependency with a '@waiting' tag and then write what I'm waiting for in the task description.

![waiting.png](waiting.png)

Although task dependency may not appear in the Todoist user interface, it's a well-known linear program. (we can define linear program here if needed). There are two models or frameworks we can use to minimize the time it takes to complete tasks when some tasks depend on others. The first is called multi-planning problem and the other is longest-path. With these two models, we can determine what order I should complete my tasks in to minimize the time it takes to do everything. 

**In this project, I'm going to model tasks with dependencies from my own to-do list. From there, I plan to use these models as a starting point for suggesting how to incorporate a task dependency feature into the Todoist app.**




The first few sentences should give a quick overview of the entire project. Then, elaborate with a description of the problem that will be solved, a brief history (with [citations](https://en.wikipedia.org/wiki/Citation)) of how the problem came about, why it's important/interesting, and any other interesting facts you'd like to talk about. You should address and explain where the problem data is coming from (research? the internet? synthetically generated?) Also give an outline of the rest of the report.

This section should be 300-600 words long, and **should be accessible to a general audience** (don't assume your reader has taken the class!). Feel free to include images if you think it'll be helpful:

For more help on using Markdown, see [this reference](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).


## 2. Mathematical models ##

Let's take a look at that to-do list again:

![todolist.png](todolist.png)

### The Traveling Salesperson Problem

The traveling salesperson problem is as follows: Suppose a salesperson has to visit a certain number of cities and must end the tour in the same city they started in. How do they travel to all cities exactly once in order to minimize the total distance traveled?

We can visualize this through a graph (insert image here)

We can formulate our to-do list problem as a traveling salesperson problem. Given a list of tasks to be done where each task must be done sequentially and a set of dependency constraints, what order should the tasks be done in order to minimize the total work time?

Unlike the traveling salesperson problem, which only wants to create one tour that covers all cities, we want to create a task list for all the tasks by splitting them into different days. In other words, we want to split the list of tasks into several "subtours", where each "subtour" is the list of tasks (in order) that need to be done each day. In this case, the fact that each subtour ends at the same task they started with doesn't exactly cross over into the original problem and can be ignored.

#### Decision Variables 

In our formulation of the Traveling Salesperson Problem, the decision variables are the day that a task is done and the order that task is in for a given day (and therefore in the order relative to all tasks). We will use two binary variables to represent these two aspects of the decision variable(s). $z_{ij}$ represents the order a task is done in, where $z_{ij}$ means that task i is followed by task j. $d_{ij}$ is a number of tasks x days matrix where $d_{ij}$ represents that task i is done on day j.

$ z_{i,j \in \mathcal{T}} = \begin{cases}
1 \quad \text{if i is followed by j} \\
0 \quad \text{otherwise}
\end{cases}$

$ d_{i \in \mathcal{T}, j \in \mathcal{D}} = \begin{cases}
1 \quad \text{if task i is completed on day j} \\
0 \quad \text{otherwise}
\end{cases}$

Let $\mathcal{T}$ be the set of tasks to complete.
Let $\mathcal{D}$ be the set of days for the tasks to be completed in.
Let $\mathcal{P}$ be the set of tasks that are the ancestors of a given task.
Let $\text{duration}_i$ be the set of task durations for a task i.
Let $\text{total_work_time}$ be the set of total work times for each day in $\mathcal{D}$.

#### Constraints

##### Tasks must be completed one at a time:
* All of the tasks must be done: $\sum_{i,j \in \mathcal{T}} z_{ij} = |\mathcal{T}|$ 
* All tasks must be completed sequentially:
    * $\sum_{j \in \mathcal{T}} z_{ij} = 1 \quad \forall i \in \mathcal{T}$
    * $\sum_{i \in \mathcal{T}} z_{ij} = 1 \quad \forall j \in \mathcal{T}$ 
* A task can't follow itself: $z_{ii} = 0$
* Once a task is completed, it can't be done again: $z_{ij} + z_{ji} \le 1$
* precedence constraints must be upheld: for all tasks in pred, $z_{ij} = 0$
    
##### Tasks are assigned to days
* Each task must be assigned a day: $\sum_{j \in \mathcal{D}} d_{ij} = 1$
* Each task can only be assigned to one day: $\sum_{i \in \mathcal{T}, j \in \mathcal{D}} d_{ij} \le |\mathcal{T}|$
* the total time to complete all the tasks in a day cannot exceed the amount of time available: $\sum_{i \in \mathcal{T}, j \in \mathcal{D}} d_{ij}*\text{duration}_{i} \le \text{total_work_time}_j$
* each task must be completed before its deadline: for i in tasks and j in days, $d_ij = 0$ when $j > deadlines_i$

#### Objective

We want to minimize the amount of time procrastinating, so we will try to complete all the tasks as soon as possible:

$\text{minimize} \sum d_{ij} * j \qquad \forall i \in \mathcal{T}, \quad \forall j \in \mathcal{D}$

Since $d_{ij}$ are binary variables, we are summing over indices for each day that work is done.

#### In standard form, we can write this as:
$$
\begin{aligned} \\
\text{minimize} \qquad& \sum d_{ij} * j \quad \forall i \in \mathcal{T}, \quad \forall j \in \mathcal{D} \\
\text{subject to:}\qquad& \sum z_{ij} = |\mathcal{T}|, \qquad {i,j \in \mathcal{T}} \\
& \sum_{j \in \mathcal{T}} z_{ij} = 1 \quad \forall i \in \mathcal{T} \\
& \sum_{i \in \mathcal{T}} z_{ij} = 1 \quad \forall j \in \mathcal{T} \\
& z_{ii} = 0 \\
& z_{ij} + z_{ji} \le 1 \\
& z_{ij} = 0 \\
& \sum_{j \in \mathcal{D}} d_{ij} = 1 \\
& \sum_{i \in \mathcal{T}, j \in \mathcal{D}} d_{ij} \le |\mathcal{T}| \\
& \sum_{i \in \mathcal{T}, j \in \mathcal{D}} \\
& d_{ij}*\text{duration}_{i} \le \text{total_work_time}_j \\
& d_{ij} = 0\qquad \text{when } j > \text{deadlines}_i \\
\end{aligned}
$$
 
For some quick tips on using $\LaTeX$, see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).

## 3. Solution ##

## Data for the problem:

In [1]:
tasks = []
# create a dictionary with names, duration, travel time

for i = 'a':'f'
    push!(tasks, Symbol(i))
end

# order of tasks:
# 524 hw, 577 hw, print off hw sheet, 354 hw, 354 project, print off 354 notes sheet

# 524 hw, 577 hw, print off hw sheet, 354 hw, 354 project, print off 354 notes sheet
# duration is expressed in minutes (where tasks have a duration in 15 minute intervals)
task_durations = [ 3*60 60 15 120 6*60 15 ]

# pred stores ancestors on each task
pred = ( [], [], [4], [], [], [], [] )
    
# Day related data:
    
# each entry corresponds to a day
available_work_time = [ 5*60 2*60 60 45 24*60]
# deadlines for each task (assume Monday is day 1) - 11:59pm that day is the same as <= deadline + 1
deadlines = [ 3 4 4 1 5 2 ]


1×6 Matrix{Int64}:
 3  4  4  1  5  2


| Task Letter | Task Description          | Immediate Predecessors | Duration (minutes) |
|-------------|---------------------------|------------------------|--------------------|
| a           | 524 hw                    |                        |                    |
| b           | 577 hw                    | c                      |                    |
| c           | print off 577 hw sheet    |                        |                    |
| d           | 354 hw                    |                        |                    |
| e           | 354 project               |                        |                    |
| f           | print off 354 notes sheet |                        |                    |

In [34]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
set_silent(m)

# sequence binary variable - maybe I can make this an integer variable
# @variable(m, z[1:length(task_durations)], Int)
# day integer variable where d[i,j] is the task i assigned to day j
@variable(m, d[1:length(task_durations), 1:length(available_work_time)], Bin)

# all tasks must be done
# each task must be assigned a day
@constraint(m, r2[i=1:length(task_durations)], sum( d[i,j] for j=1:length(available_work_time)) == 1)

# only one day per task
@constraint(m, sum(d[i,j] for i=1:length(task_durations), j=1:length(available_work_time) ) <= length(task_durations ))   

# the time to complete all tasks in a day can't exceed work time per day
[ @constraint(m, sum(d[i,j]*task_durations[i] for i=1:length(task_durations)) <= available_work_time[j]) for j=1:length(available_work_time) ]

# enforce precedence constraints in d - a task q that follows p cannot happen on a day before p is done
for q=1:length(task_durations)
    for p in pred[q]
        for j=1:length(available_work_time)
            if (d[p,j] == 1)
                for k=1:j-1
                    @constraint(m, d[q,k] == 0)
                end
            end
        end
    end
end
          
# set deadlines
[ @constraint(m, d[i,j] == 0) for i=1:length(task_durations), j=1:length(available_work_time) if j > deadlines[i]]

# we want to prevent procrastination, so we are going to minimize the last day that all the tasks can be done
@objective(m, Min, sum( d[i,j]*j for i=1:length(task_durations), j=1:length(available_work_time) ) )

optimize!(m)

println("Objective value: ", objective_value(m))


D = value.(d)
println(D)
for i=1:length(task_durations)
    for j=1:length(available_work_time)
        if (D[i,j] == 1)
            println("Task ",i," is done on day ", j)
        end
    end
end

for j=1:length(available_work_time)
    println("Sum of day ",j,": ",sum( D[i,j]*task_durations[i] for i=1:length(task_durations) ) )
    println(available_work_time[j])
end

# print out output in a meaningful way:
# for each day, 
# print out the order of the tasks

# find the first task for each day, then work through z to get the order of the tasks for that day

for j=1:length(available_work_time)
    println("Day ",j,": ")
    for i=1:length(task_durations)
        if (D[i,j] == 1)
            println("Task ",i," is done on day ", j)
        end
    end
    # println("Sum of day ",j,": ",sum( D[i,j]*task_durations[i] for i=1:length(task_durations) ) )
    # println(available_work_time[j])
end

# overall sequencing model given D - put all tasks in a given order
m2 = Model(HiGHS.Optimizer)
set_silent(m2)
# sequence binary variable - maybe I can make this an integer variable
@variable(m2, z[1:length(task_durations)], Int)

for i=1:length(z)
    @constraint(m2, z[i] >= 1)
    @constraint(m2, z[i] <= length(z))
end

for q=1:length(task_durations)
    for p in pred[q]
        # @constraint(m2, z[q] - z[p] >= 1)
    end
end

for i=1:length(task_durations)
    for j=1:length(available_work_time)
        if (D[i,j] == 1)
            for l=1:length(task_durations)
                for k=(j+1):length(available_work_time)
                    if (D[l,k] == 1)
                        @constraint(m2, z[i] <= z[l])
                    end
                end
            end
        end
    end
end

# if there is more than one task on the same day, compare by deadline
"""
for j=1:length(available_work_time)
    if ( sum(D[i,j] for i=1:length(task_durations) ) > 1)
        for i=1:length(task_durations)
            
    end
end
"""       
    
for i=1:length(z)
    for j=1:length(z)
        if (i != j)
            # they must differ by at least 1
            # @constraint(m2, z[i]-z[j] >= 0)
        end
    end
end
    
# @constraint(m2, sum(z) == sum(1:length(z)))

@objective(m2, Min, sum(z[i]*deadlines[i] for i=1:length(z)) )

optimize!(m2)

Z = value.(z)
println(Z)

Objective value: 13.0
[1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0; 0.0 1.0 0.0 0.0 0.0]
Task 1 is done on day 1
Task 2 is done on day 2
Task 3 is done on day 2
Task 4 is done on day 1
Task 5 is done on day 5
Task 6 is done on day 2
Sum of day 1: 300.0
300
Sum of day 2: 90.0
120
Sum of day 3: 0.0
60
Sum of day 4: 0.0
45
Sum of day 5: 360.0
1440
Day 1: 
Task 1 is done on day 1
Task 4 is done on day 1
Day 2: 
Task 2 is done on day 2
Task 3 is done on day 2
Task 6 is done on day 2
Day 3: 
Day 4: 
Day 5: 
Task 5 is done on day 5


LoadError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [10]:
D = value.(d)
Z = value.(z)
println(D)
println(Z)

# iterate through D, make a list of each task with its day (dictionary?)
# then for each day, go to that dictionary
# search through Z 
# find first task in Z - is there a first task?

┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP ~/.julia/packages/JuMP/yYfHy/src/optimizer_interface.jl:712


LoadError: OptimizeNotCalled()

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

## Ideas:
Bar graphs indicating how much time per day is used for work times
Graphs illustrating what the clusters are?

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.

## Citations

### Modeling, writing, etc.

### formulating the project itself:
ChatGPT - one line constraints